### EOD data extraction ETL process

The following ETL process will extract data from [eodhistoricaldata.com](https://eodhistoricaldata.com/) API. This API offers historical and end-of-day financial market data for various asset classes, including stocks, indices, options, futures, currencies, and cryptocurrencies. The data provided may include historical prices, trading volumes, fundamental financial data, splits, dividends, and more.

In [17]:
#Importing required libraries
import pandas as pd
import numpy as np
import os
from io import StringIO
from typing import Optional
import requests
import awswrangler as wr
import gspread
from gspread_dataframe import set_with_dataframe
from utilities import init_session, sanitize_dates, format_date, url, RemoteDataError

#### Creating S3 bucket
This step assumes that you have configured your AWS CLI locally, if this is not the case you can follow this link in order to configure your [AWS Command Line Interface](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html).

You can run the cell above or you can also execute the following line in your command line `aws s3api create-bucket --bucket eod-data-test-bucket --region us-east-2 --create-bucket-configuration LocationConstraint=us-east-2`, it will create a new bucket in your S3 storage named `eod-data-test-bucket` or any other name you prefer, just please follow this link that will guide you on the naming rules for buckets [Bucket naming rules](https://docs.aws.amazon.com/AmazonS3/latest/userguide/bucketnamingrules.html)

In [ ]:
!aws s3api create-bucket --bucket eod-data-test-bucket --region us-east-2 --create-bucket-configuration LocationConstraint=us-east-2

#### Setting local variables

In this step is necessary that you register for a free account on [eodhistoricaldata.com](https://eodhistoricaldata.com/), once registered go to the settings page to get access token. For free accounts only 20 API calls are allowed  per days, which will be more than enough for this ETL process.

Once your token is set is recommended to store tokens and access keys in your environment variables, in the case you are working in a Windows or a Linux environment. Then you can use the `os` python module in order to call those credentials to your script.

I heve previously stored my credentials as environment variables in Windows by running `setx [VARIABLE_NAME] "VARIABLE_VALUE"` in the Command Prompt or follow this [link](https://geekflare.com/system-environment-variables-in-windows/) for guidance.

In [2]:
#Setting variables
token = os.getenv('EOD_API_TOKEN')
aws_key = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret = os.environ.get('AWS_SECRET_ACCESS_KEY')
#This API key is used for demo purposes
EOD_HISTORICAL_DATA_API_KEY_DEFAULT = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"
EOD_HISTORICAL_DATA_API_URL = "https://eodhistoricaldata.com/api"
#This is the bucket name that was created in your AWS account
bucket = 'eod-datalake'

##### Helper functions

These functions will help to get data from the EOD API and once the data is pulled from the API, we will store it in S3 bucket and in Google Sheets for data visualization purposes.
 
There are some endpoints to get data from EOD, [fundamentals](https://eodhd.com/financial-apis/stock-etfs-fundamental-data-feeds/), [real-time](https://eodhd.com/financial-apis/live-realtime-stocks-api/), [end of day](https://eodhd.com/financial-apis/api-for-historical-data-and-volumes/), [intraday](https://eodhd.com/financial-apis/intraday-historical-data-api/) and [Web Socket Connection](https://eodhd.com/financial-apis/new-real-time-data-api-websockets/). Some of them are available for the free tier, and we will use for this demo the `eod` endpoint.

In [18]:
def get_eod_data(symbol:str,
                 exchange:str,
                 endpoint:str,
                 start:Optional[str] = None,
                 end:Optional[str] = None,
                 api_key=token,
                 session=None):
    """
    Returns EOD (end of day data) for a given symbol
    """
    symbol_exchange = symbol + "." + exchange
    session = init_session(session)
    start, end = sanitize_dates(start, end)
    endpoint = f"/{endpoint}/{symbol_exchange}"
    url = EOD_HISTORICAL_DATA_API_URL + endpoint
    params = {
        "api_token": api_key,
        "from": format_date(start),
        "to": format_date(end)
    }
    
    r = session.get(url, params=params)
    
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), 
                         skipfooter=1,
                         parse_dates=[0], 
                         index_col=0)
        return df
    
    else:
        params["api_token"] = "YOUR_HIDDEN_API"
        raise RemoteDataError(r.status_code, r.reason, url(url, params))
        
def get_fundamental_data(symbol: str,
                         exchange: str,
                         endpoint: str,
                         api_key=token,
                         session=None):
    """
    Returns EOD (end of day data) for a given symbol
    """
    symbol_exchange = symbol + "." + exchange
    session = init_session(session)
    endpoint = f"/{endpoint}/{symbol_exchange}"
    url = EOD_HISTORICAL_DATA_API_URL + endpoint
    params = {
        "api_token": api_key
    }
    
    r = session.get(url, params=params)
    
    if r.status_code == requests.codes.ok:
        #df = pd.read_csv(StringIO(r.text), skipfooter=1,
        #                 parse_dates=[0], index_col=0)
        print(r.text)
    else:
        pass

    return r.text

def write_data_to_bucket(file_name:str, mode:str):
    """
    Parameters:
    ----------
    mode(str): Available write modes are 'append', 'overwrite' and 'overwrite_partitions'
    """

    path = f"s3://{bucket}/raw-data/{file_name}"
    #Sending dataframe of corresponding ticker to bucket
    wr.s3.to_csv(
        df=df,
        path=path,
        index=True,
        dataset=True,
        mode=mode
    )

def read_csv_from_bucket(folder_name:str) -> pd.DataFrame:

    df = wr.s3.read_csv(path = f"s3://{bucket}/raw-data/{folder_name}/",
                        path_suffix = ".csv"
)
    return df

In [29]:
symbols = ['AAPL','AMZN']

In [30]:
#Stores in a dataframe data for a single ticker
df1 = get_eod_data(symbols[0], "US", "eod")
print(f"This dataset has {df1.shape[0]} rows")
df1.head()

This dataset has 250 rows


C:\Users\felix\AppData\Local\Temp\ipykernel_7552\3566820975.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(r.text),


,Open,High,Low,Close,Adjusted_close,Volume
Date,,,,,,
2022-08-08,166.37,167.81,164.20,164.87,164.1190,60276900
2022-08-09,164.02,165.82,163.25,164.92,164.1688,63135500
2022-08-10,167.68,169.34,166.90,169.24,168.4691,70170500
2022-08-11,170.06,170.99,168.19,168.49,167.7225,57149200
2022-08-12,169.82,172.17,169.40,172.10,171.3161,68039400


In [31]:
#Stores in a dataframe data for a single ticker
df2 = get_eod_data(symbols[1], "US", "eod")
print(f"This dataset has {df2.shape[0]} rows")
df2.head()

This dataset has 250 rows


C:\Users\felix\AppData\Local\Temp\ipykernel_7552\3566820975.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(r.text),


,Open,High,Low,Close,Adjusted_close,Volume
Date,,,,,,
2022-08-08,142.050,144.2300,138.2900,139.41,139.41,52303480
2022-08-09,138.052,138.9523,136.2100,137.83,137.83,40434719
2022-08-10,142.900,144.6000,141.0100,142.69,142.69,54773820
2022-08-11,143.860,144.4900,139.7600,140.64,140.64,44867340
2022-08-12,142.050,143.5700,140.1201,143.55,143.55,47643480


In [41]:
# List of dataframes
df_list = [df1, df2]

In [43]:
# Create the 'processed_data' folder if it doesn't exist
if not os.path.exists('processed_data'):
    os.makedirs('processed_data')

# Loop through the list of DataFrames and save them as CSV files
for i, df in enumerate(df_list, start=1):
    file_name = f'df_{i}.csv'
    file_path = os.path.join('processed_data', file_name)
    df.to_csv(file_path, index=False)

print("DataFrames saved as CSV files in the 'processed_data' folder.")

DataFrames saved as CSV files in the 'processed_data' folder.
